In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter 

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
urls = [
    "https://www.langchain.com/breakoutagents",
    "https://blog.langchain.com/langchain-state-of-ai-2024"
]

In [5]:
docs = [WebBaseLoader(url).load() for url in urls]
docs

[[Document(metadata={'source': 'https://www.langchain.com/breakoutagents', 'title': 'Breakout Agentic Apps', 'description': 'Dive into the stories of companies pushing the boundaries of AI agents. Learn "why" and "how" they made specific architecture, UX, prompt engineering, and evaluation choices for high-impact results.', 'language': 'en'}, page_content="Breakout Agentic Apps\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProducts\n\nFrameworksLangGraphLangChainPlatformsLangSmithLangGraph PlatformResources\n\nGuidesBlogCustomer StoriesLangChain AcademyCommunityEventsChangelogDocs\n\nPythonLangGraphLangSmithLangChainJavaScriptLangGraphLangSmithLangChainCompany\n\nAboutCareersPricingGet a demoSign upBreakout Agentic AppsIn this series, dive into the stories of companies pushing the boundaries of AI agents. Learn how each team approached:\u200d•\xa0UX:\xa0How users interact with their agent• Cognitive architecture: How their agent thinks•\xa0Prompt engineering: Best practices for prompting• E

In [6]:
doc_list = [item for sublist in docs for item in sublist]

"""
doc_list = []
for sublist in docs:
    for item in sublist:
        doc_list.append(item)
"""

'\ndoc_list = []\nfor sublist in docs:\n    for item in sublist:\n        doc_list.append(item)\n'

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

doc_splits = text_splitter.split_documents(doc_list)

print(doc_splits[:2])

[Document(metadata={'source': 'https://www.langchain.com/breakoutagents', 'title': 'Breakout Agentic Apps', 'description': 'Dive into the stories of companies pushing the boundaries of AI agents. Learn "why" and "how" they made specific architecture, UX, prompt engineering, and evaluation choices for high-impact results.', 'language': 'en'}, page_content="Breakout Agentic Apps\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProducts\n\nFrameworksLangGraphLangChainPlatformsLangSmithLangGraph PlatformResources\n\nGuidesBlogCustomer StoriesLangChain AcademyCommunityEventsChangelogDocs\n\nPythonLangGraphLangSmithLangChainJavaScriptLangGraphLangSmithLangChainCompany\n\nAboutCareersPricingGet a demoSign upBreakout Agentic AppsIn this series, dive into the stories of companies pushing the boundaries of AI agents. Learn how each team approached:\u200d•\xa0UX:\xa0How users interact with their agent• Cognitive architecture: How their agent thinks•\xa0Prompt engineering: Best practices for prompting• Ev

In [9]:
print(doc_splits[0])

page_content='Breakout Agentic Apps




















Products

FrameworksLangGraphLangChainPlatformsLangSmithLangGraph PlatformResources

GuidesBlogCustomer StoriesLangChain AcademyCommunityEventsChangelogDocs

PythonLangGraphLangSmithLangChainJavaScriptLangGraphLangSmithLangChainCompany

AboutCareersPricingGet a demoSign upBreakout Agentic AppsIn this series, dive into the stories of companies pushing the boundaries of AI agents. Learn how each team approached:‍• UX: How users interact with their agent• Cognitive architecture: How their agent thinks• Prompt engineering: Best practices for prompting• Evaluations: How to gain confidence in agent performance


Building an AI tour guide that helps users navigate Ramp's platform for financial operations


Navigate your inbox and calendar in a flash, with an AI-powered search assistant for emails


An AI answer engine that lets you handle complex query searches like a (Perplexity) Pro' metadata={'source': 'https://www.langchain.com/break

In [11]:
# add all text to vector DB
vector_store = FAISS.from_documents(
    documents=doc_splits,
    embedding=OpenAIEmbeddings()
)

retriever = vector_store.as_retriever()

In [12]:
retriever.invoke("What is LangGraph?")

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


[Document(id='a11f71ce-8263-4114-8ff2-a046d7eabbdd', metadata={'source': 'https://blog.langchain.com/langchain-state-of-ai-2024', 'title': 'LangChain State of AI 2024 Report', 'description': 'Dive into LangSmith product usage patterns that show how the AI ecosystem and the way people are building LLM apps is evolving.', 'language': 'en'}, page_content='for 15.3% of LangSmith usage this year, increasing 3x compared to the previous year.\xa0AI agents are gaining tractionAs companies are getting more serious about incorporating AI agents across various industries, adoption of our controllable agent framework, LangGraph, is also on the rise. Since its release in March 2024, LangGraph has steadily gained traction —\xa0with 43% of LangSmith organizations are now sending LangGraph traces. These traces represent complex, orchestrated tasks that go beyond basic LLM interactions.This growth aligns with the rise in agentic behavior: we see that on average 21.9% of traces now involve tool calls, u

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [13]:
## convert retriever to RETRIEVER TOOLS
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retriever_vector_db_blog",
    "A tool to search the LangGraph blog for information"
)

In [14]:
retriever_tool

Tool(name='retriever_vector_db_blog', description='A tool to search the LangGraph blog for information', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000016B1F25CEA0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000016B21927560>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000016B1F25CFE0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000016B21927560>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_con

In [15]:
## for langchain Blogs

langchain_urls = [
    "https://blog.langchain.com/agent-middleware/",
    "https://blog.langchain.com/standard-message-content/"
]

docs = [WebBaseLoader(url).load() for url in langchain_urls]
docs


[[Document(metadata={'source': 'https://blog.langchain.com/agent-middleware/', 'title': 'Agent Middleware', 'language': 'en'}, page_content='\n\n\nAgent Middleware\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAgent Middleware\n\n5 min read\nSep 8, 2025\n\n\n\n\n\nLangChain has had agent abstractions for nearly three years. There are now probably 100s of agent frameworks with the same core abstraction. They all suffer from the same downsides that the original LangChain agents suffered from: they do not give the developer enough control over context engineering when needed, leading to developers graduating off of the abstraction for any non-trivial use case. In LangChain 1.0 we are introducing a new agent abstraction (Middleware) whic

In [16]:
doc_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

doc_splits = text_splitter.split_documents(doc_list)

# Add all these text to DB

vectorstorelangchain = FAISS.from_documents(
    documents=doc_splits,
    embedding=OpenAIEmbeddings()
)

retrieverlangchain = vectorstorelangchain.as_retriever()

In [17]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool_langchain = create_retriever_tool(
    retrieverlangchain,
    "retriever_vector_db_langchain_blog",
    "A tool to search the Langchain blog for information"
)

In [18]:
tools = [retriever_tool, retriever_tool_langchain]

In [19]:
## Langgraph workflow
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

class AgenticState(TypedDict):
    # The add_messages fn defines how an update should be processed
    # Default is to replace, add_messages says "append"

    messages: Annotated[Sequence[BaseMessage], add_messages]

In [20]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CQJIi7c78xCTjzntnGqsUxGBxvHGo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--44e16e9f-c6db-4852-96eb-e0d1fc244292-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [21]:
def agent(state: AgenticState):
    """
    Invokes the agent model to generate a response based on the current static.
    Given the question, it will decide to retrive using the retriever tool, or simply end.
    
    Args:
        state (messages): The current state of the conversation.

    Returns:
        dict: The updated state with the agent's response to messages.
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    model = model.bind_tools(tools)
    response = model.invoke(messages)
    # we return a list, because this will get added to the existing list
    return {"messages": [response]}
        



In [23]:
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field

from typing import Literal, Annotated, Sequence
from typing_extensions import TypedDict

In [26]:
def grade_documents(state: AgenticState) -> Literal["generate", "rewrite"]:
    """
    Determines whether the retrieved docs are relevant to the question.

    Args:
        state (messages): The current state of the conversation.
    
    Returns:
        str: A decision for whether the docs are relevant or not
    """
    print("---CHECK RELEVANCE---")

    # Data Model
    class grade(BaseModel):
        """Binary score for relevance check"""

        binary_score: str = Field(description="relevance score 'yes' or 'no'")

    # LLM
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    #LLM with tool & validation
    llm_with_tool = model.with_structured_output(grade)

    # Prompt
    prompt = PromptTemplate(
        template=""" You are a grader assessing relevance of a retrieved documents to a user.
        Here is the retrieved documents: \n\n {context} \n\n
        Here is the user question: \n\n {question} \n\n
        If the documents contains keyword(s) or semantic meaning related to the user question,
        Give a binary score 'yes' or 'no' to indicate whether the document is relevant or not
        """,
        input_variables = ["context", "question"]
    )

    # chain
    chain = prompt | llm_with_tool

    messages = state["messages"]
    last_message = messages[-1]

    question = messages[0].content
    docs = last_message.content

    scored_result = chain.invoke({"context": docs, "question": question})

    score = scored_result.binary_score

    if score == "yes":
        print("---RELEVANT---")
        return "generate"

    else:
        print("---NOT RELEVANT---")
        print(score)
        return "rewrite"



In [27]:
def generate(state: AgenticState):
    """
    Generates a response using the retrieved documents.

    Args:
        state (messages): The current state of the conversation.

    Returns:
        dict: The updated state with the generated response to messages.
    """
    
    print("---GENERATE---")
    messages = state["messages"]
    question = messages[0].content
    last_message = messages[-1]

    docs = last_message.content

    # Prompt
    prompt = hub.pull("rlm/rag-prompt")

    # LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # chain
    rag_chain = prompt | llm| StrOutputParser()

In [28]:
def rewrite(state: AgenticState):
    """
    transform the query to produce a better question.

    Args: 
        state (messages): The current state of the conversation.
    
    Returns:
        dict: The updated state with the rewritten question to messages.
    """
    print("---TRANSFORM QUERY---")
    messages = state["messages"]
    question = messages[0].content

    msg = [
        HumanMessage(
            content=f""" \n
    Look at the input & try to reason about the underlying semantic intent/meaning.
    Here is the initial question:
    \n-----------\n
    Formulate an improved question:""",

        )
    ]

    # Grader
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    response = model.invoke(msg)
    return {"messages": [response]}

In [29]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

# define a graph
workflow = StateGraph(AgenticState)

# define the nodes
workflow.add_node("agent", agent)
retrieve = ToolNode([retriever_tool, retriever_tool_langchain])
workflow.add_node("retrieve", retrieve)
workflow.add_node("rewrite", rewrite)
workflow.add_node("generate", generate) # generate a response after we know the docs are relevant

workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    tools_condition,
    {
        # translate the condition outputs to nodes in our graph
        "tools": "retrieve",
        END: END, 
    },
)

# Edges taken after action node is called
workflow.add_conditional_edges("retrieve",
    # Assess the agent decision
    grade_documents,
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")
    
# compile the graph
graph = workflow.compile()

# run the graph
graph.invoke({"messages": [HumanMessage(content="What is LangGraph?")]})


---CALL AGENT---
---CHECK RELEVANCE---
---RELEVANT---
---GENERATE---


{'messages': [HumanMessage(content='What is LangGraph?', additional_kwargs={}, response_metadata={}, id='d8b5739d-b008-4206-8577-c9b82b1f7205'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GRlwejI0hBK8z3C94wyWYBzG', 'function': {'arguments': '{"query":"What is LangGraph?"}', 'name': 'retriever_vector_db_blog'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 104, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CQK6HzMKmaSDzRMZVJSCKDmRV2ET3', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8eba4555-4738-47f1-bbb5-31838b43c7a0-0', tool_calls=[{'name': 'retriever_vector_db_blog', 'args': 